In [3]:
import json
import pandas as pd
import nltk
import string
import re
import csv
from matplotlib import pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer, LancasterStemmer
nltk.data.path.append("/Users/youqiao/workspace/env/nltk_data")
%matplotlib inline

In [4]:
questions = pd.read_csv('Questions.csv',encoding='latin1')
questions["content"] = questions["Title"] + " " + questions["Body"]
tags = pd.read_csv('Tags.csv', encoding='latin1')
stop = stopwords.words('english')

In [25]:
for k in keyword.kwlist:
    stop.append(k)

In [27]:
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

def cleantags(text):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', text)
    return cleantext

def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed

def lemmatize_tokens(tokens, lemmatizer):
    lemmatized = []
    for item in tokens:
        lemmatized.append(lemmatizer.lemmatize(item, pos='v'))
    return lemmatized

def extractwords(tokens):
    new_tokens = []
    for word, pos in nltk.pos_tag(tokens):
        if pos[:2] == 'NN' or pos[:2] == "JJ":
            new_tokens.append(word)
    return new_tokens

def excludestopwords(tokens):
    new_tokens = []
    for word in tokens:
        if word not in stopwords:
            new_tokens.append(word)
    return new_tokens

def tokenize_language(text):
    text = cleantags(text)
    translator = str.maketrans('', '', string.punctuation)
    no_punctuation = text.translate(translator)
    tokens = nltk.word_tokenize(no_punctuation)
    return tokens

def tokenize(text):
    text = cleantags(text)
    translator = str.maketrans('', '', string.punctuation)
    no_punctuation = text.translate(translator)
    tokens = nltk.word_tokenize(no_punctuation)
    lemmatizes = lemmatize_tokens(tokens, lemmatizer)
#     return extractwords(lemmatizes)
    return lemmatizes

def tokenizewithnostem(text):
    text = cleantags(text)
    translator = str.maketrans('', '', string.punctuation)
    no_punctuation = text.translate(translator)
    tokens = nltk.word_tokenize(no_punctuation)
    lemmatizes = lemmatize_tokens(tokens, lemmatizer)
    return lemmatizes

def getitemset(row, wordbags):
    itemsets = []
    for word in wordbags:
        if row[word] == 1:
            itemsets.append(word)
    return ",".join(itemsets)


class AssociationRule():
    def __init__(self):
        self.wordbag = None
        self.df = questions
        self.df_tfidf = None
        self.vector = None

    def preprocess(self, tag):
        """
        :param tag: str
        :return:
        """
        ids_tag = tags[tags["Tag"] == tag]["Id"]
        self.df = pd.DataFrame(ids_tag)
        self.df = self.df.set_index('Id').join(questions.set_index('Id'))
        print(self.df.shape)

    def tokenizing(self, ngram=1, min_df=0.1, max_df=0.9, tokenizef=None, voc=None):
        self.df = self.df.sample(n=10000, random_state=36)
        tfidfvector = TfidfVectorizer(tokenizer=tokenizef, ngram_range=(1, ngram), min_df=min_df, max_df=max_df, stop_words=stop, vocabulary=voc)
        text_tfidf = tfidfvector.fit_transform(self.df["content"])
        print(text_tfidf.shape)
        self.wordbag = tfidfvector.get_feature_names()
        self.df_tfidf = text_tfidf.toarray()
        self.df_tfidf = pd.DataFrame(self.df_tfidf, columns=self.wordbag)

        for word in self.wordbag:
            self.df_tfidf.ix[self.df_tfidf[word] == 0, word] = 0
            self.df_tfidf.ix[self.df_tfidf[word] != 0, word] = 1
            self.df_tfidf[word] = self.df_tfidf[word].astype(int)

    def genitemset(self):
        self.df_tfidf["itemset"] = self.df_tfidf.apply(lambda row: getitemset(row, self.wordbag), axis=1)
        basketfile = open("stackoverflow.basket", "w")
        for p in pd.DataFrame(self.df_tfidf["itemset"]).values:
            s1 = p[0].replace("\"", "")
            basketfile.write(s1 + "\n")
        basketfile.close()

    def genarff(self):
        self.df_tfidf.to_csv("stackoverflow.csv", header=False, index=False)

        arfffile = open("stackoverflow.arff", "w")
        arfffile.write("@relation stackoverflow.data\n\n")
        for s in self.wordbag:
            s1 = s.replace(" ", "_")
            arfffile.write("@attribute " + s1 + " {0, 1}\n")
        arfffile.write("\n")
        arfffile.write("@data\n")
        arfffile.close()

# Example for generating itemset for python to process

In [16]:
ar2 = AssociationRule()
ar2.preprocess("pandas")
ar2.tokenizing(ngram=1, min_df=0.1, max_df=0.99, tokenizef=tokenize, voc=None)
ar2.genitemset()

(2000, 6)
(2000, 52)


/Users/youqiao/env/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:93: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


In [31]:
ar2.df_tfidf

,0,1,10,2,3,4,5,6,7,8,...,try,two,use,value,want,way,work,would,x,itemset
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,"base,data,get,last,list,need,python,thank,value"
1,1,1,0,1,0,0,0,0,0,0,...,0,1,1,1,0,0,0,0,0,"0,1,2,base,data,example,get,give,however,index..."
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,"column,dataframe,date,first,follow,index,last,..."
3,0,0,0,1,0,0,0,0,0,0,...,1,1,1,0,0,0,0,0,0,"2,b,c,could,create,data,date,different,get,how..."
4,1,1,1,1,1,1,1,1,1,1,...,1,0,1,1,1,1,1,0,1,"0,1,10,2,3,4,5,6,7,8,9,also,b,c,columns,data,d..."
5,1,1,0,1,1,0,0,0,0,0,...,1,1,0,1,0,0,0,0,0,"0,1,2,3,column,contain,data,error,first,follow..."
6,1,1,0,1,1,0,0,0,0,0,...,1,1,0,1,0,0,0,0,0,"0,1,2,3,call,columns,contain,convert,dataframe..."
7,0,0,0,0,0,0,0,0,0,0,...,0,0,1,1,1,1,0,1,0,"also,base,column,csv,data,date,df,example,file..."
8,0,0,0,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,"3,call,error,file,find,get,import,last,line,li..."
9,0,0,1,0,0,0,0,0,0,1,...,0,1,1,0,0,0,0,0,1,"10,8,column,dataframe,def,find,function,index,..."


# Example of extracting those question with tag of django

In [28]:
ar3 = AssociationRule()
ar3.preprocess("django")
ar3.tokenizing(ngram=1, min_df=0.1, max_df=0.99, tokenizef=tokenize, voc=None)
ar3.genarff()

(62818, 6)
(10000, 76)


/Users/youqiao/env/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:94: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


# Example of using given dictionary to convert text

In [29]:
languages = {"javascript": 0,"java": 1,
             "php": 2,"css": 3,"ruby": 4,
             "c": 5,"swift": 6,"scala": 7,
             "r": 8,"matlab": 9,"python": 10}
ar4 = AssociationRule()
ar4.tokenizing(ngram=1, min_df=0.1, max_df=0.99, tokenizef=tokenize_language, voc=languages)
ar4.genarff()

(607282, 11)


/Users/youqiao/env/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:92: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


# Python packages

In [18]:
packages = {"requests": 0, "scrapy": 1, "wxpython": 2, "pillow": 3, 
            "sqlalchemy": 4, "beautifulsoup": 5, "twisted": 6, "numpy": 7, "scipy": 8, "matplotlib": 9,
            "pygame": 10, "pyglet": 11, "pyqt": 12, "pygtk": 13, "scapy": 14, "pywin32": 15, 
            "nltk": 16, "nose": 17, "sympy": 18, "ipython": 19, "sklearn": 20}
ar5 = AssociationRule()
ar5.tokenizing(ngram=1, min_df=0.1, max_df=0.99, tokenizef=tokenize_language, voc=packages)
ar5.genarff()

(607282, 21)


/Users/youqiao/env/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:94: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


In [21]:
import keyword

In [22]:
kw = {}
i = 0
for k in keyword.kwlist:
    kw[k] = i
    i += 1
kw

{'False': 0,
 'None': 1,
 'True': 2,
 'and': 3,
 'as': 4,
 'assert': 5,
 'break': 6,
 'class': 7,
 'continue': 8,
 'def': 9,
 'del': 10,
 'elif': 11,
 'else': 12,
 'except': 13,
 'finally': 14,
 'for': 15,
 'from': 16,
 'global': 17,
 'if': 18,
 'import': 19,
 'in': 20,
 'is': 21,
 'lambda': 22,
 'nonlocal': 23,
 'not': 24,
 'or': 25,
 'pass': 26,
 'raise': 27,
 'return': 28,
 'try': 29,
 'while': 30,
 'with': 31,
 'yield': 32}

In [23]:
ar6 = AssociationRule()
ar6.tokenizing(ngram=1, min_df=0.1, max_df=0.99, tokenizef=tokenize_language, voc=kw)
ar6.genarff()

(607282, 33)


/Users/youqiao/env/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:94: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
